# Lesson 7: ReAct

This Jupyter notebook demonstrates a practical implementation of the ReAct (Reasoning and Acting) framework using Google's Gemini model. It includes three main components: a mock search tool for demonstration, a thought generation phase using structured outputs, and an action phase with function calling capabilities. The notebook implements a complete ReAct control loop that alternates between thinking, acting, and observing, allowing the AI agent to break down complex problems iteratively.

## Setup

First, let's install the necessary Python libraries using pip.

In [1]:
!pip install -q google-genai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Configure Gemini API Key

To use the Gemini API, you need an API key. 

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  Create a file named `.env` in the root of this project.
3.  Add the following line to the `.env` file, replacing `your_api_key_here` with your actual key:
    ```
    GOOGLE_API_KEY="your_api_key_here"
    ```
The code below will load this key from the `.env` file.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lessons.utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])

Trying to load environment variables from `/Users/fabio/Desktop/course-ai-agents/.env`
Environment variables loaded successfully.


## Setting Up the Environment and the LLM Client

Let's start by importing the necessary libraries for our ReAct agent implementation:

In [4]:
from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import List, Union
from enum import Enum

from lessons.utils import pretty_print

# Create Gemini client
client = genai.Client()

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


### Define Constants

We will use the `gemini-2.5-flash` model, which is fast, cost-effective, and supports advanced features like tool use.

In [5]:
MODEL_ID = "gemini-2.5-flash"

## Search Tool Definition

Let's implement our mock search tool that will serve as the external knowledge source for our agent. This simplified version focuses on the ReAct mechanics rather than real API integration:

In [6]:
def search(query: str) -> str:
    """
    Search for information about a specific topic or query.

    Args:
        query (str): The search query or topic to look up

    Returns:
        str: Search results containing information about the queried topic

    Note:
        This is a simple mocked search tool for demonstration purposes.
        In a real scenario, this would call a search API like Google Search,
        Bing Search, or a specialized knowledge base API.
    """
    query_lower = query.lower()

    # Predefined responses for demonstration
    if all(word in query_lower for word in ["capital", "france"]):
        return "Paris is the capital of France and is known for the Eiffel Tower."
    elif "react" in query_lower:
        return "The ReAct (Reasoning and Acting) framework enables LLMs to solve complex tasks by interleaving thought generation, action execution, and observation processing."

    # Generic response for unhandled queries
    return f"Mock search result: Information about '{query}' was not found in the predefined mock responses. A real search tool would provide more."

## Thought Phase

Now let's implement the thought generation phase using structured outputs. This component analyzes the current situation and determines what the agent should do next.

In [7]:
SYSTEM_PROMPT_THOUGHT = """
Your task is to break down the user's question into smaller, answerable information retrieval steps and create a plan to gather all necessary data.

1.  What information is needed to answer the question? (Break down into granular pieces.)
2.  What is the plan to retrieve this information, step-by-step? (Focus on what information to get, not how to get it.)

Prioritize external information retrieval for all factual data. Never trust your internal knowledge.
""".strip()


PROMPT_TEMPLATE_THOUGHT = """
Conversation so far:

<conversation>
{conversation}
</conversation>

What is your thought about the next step?
""".strip()


class ThoughtResponse(BaseModel):
    thought: str


def generate_thought(conversation: str) -> str:
    """Generate a thought using structured output"""
    prompt = PROMPT_TEMPLATE_THOUGHT.format(conversation=conversation)

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_PROMPT_THOUGHT,
            response_mime_type="application/json",
            response_schema=ThoughtResponse
        )
    )
    return response.parsed.thought

## Action Phase

Next, let's implement the action phase using function calling. This component determines whether to use a tool or provide a final answer.

In [8]:
SYSTEM_PROMPT_ACTION = """
Prioritize external information retrieval for all factual data. Never trust your internal knowledge.
""".strip()


PROMPT_TEMPLATE_ACTION = """
Conversation so far:

<conversation>
{conversation}
</conversation>

Based on your thought, what action should you take?
""".strip()


# Modeling the "finish" action
ACTION_FINISH = "finish"
class ActionFinishObject(BaseModel):
    text: str


# A mapping of tool names to their functions
TOOL_REGISTRY = {
    search.__name__: search,
}


def generate_action(conversation: str) -> tuple[str, Union[dict, ActionFinishObject]]:
    """Generate an action using function calling or direct text response"""
    prompt = PROMPT_TEMPLATE_ACTION.format(conversation=conversation)

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_PROMPT_ACTION,
            tools=[search],
            automatic_function_calling={'disable': True}
        )
    )

    # Check if response contains a function call or text
    response_part = response.candidates[0].content.parts[0]

    if hasattr(response_part, 'function_call') and response_part.function_call:
        function_call = response_part.function_call
        action_name = function_call.name
        action_params = dict(function_call.args)
        return action_name, action_params
    else:
        # It's a text response (final answer)
        return ACTION_FINISH, ActionFinishObject(text=response.text)

## ReAct Control Loop

Now let's create the main ReAct control loop that orchestrates the thought-action-observation cycle. Let's model each step of the ReAct loop first.

In [9]:
class MessageRole(str, Enum):
    """Enumeration for the different roles a message can have."""
    USER = "user"
    THOUGHT = "thought"
    TOOL_REQUEST = "tool request"
    OBSERVATION = "observation"
    FINAL_ANSWER = "final answer"


class Message(BaseModel):
    """A message with a role and content, used for all message types."""
    role: MessageRole
    content: str

    def __str__(self) -> str:
        """Provides a user-friendly string representation of the message."""
        return f"{self.role.value.capitalize()}: {self.content}"

Then, let's prepare a function that generates the final answer in case that the ReAct loops has reached its maximum number of iterations.

In [10]:
PROMPT_TEMPLATE_FINAL_ANSWER = """
<conversation>
{conversation}
</conversation>

Given the conversation above, write a final answer to the original question.
""".strip()


def generate_final_answer(conversation: str) -> str:
    prompt = PROMPT_TEMPLATE_FINAL_ANSWER.format(conversation=conversation)
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt
    )
    return response.text

We'll manage a list of messages, where each message is a ReAct step, in a variable called `scratchpad` (as it was called this way originally in the ReAct paper). The following helper function converts this list of messages to a string representation that we can use in the prompts.

In [11]:
def format_scratchpad_for_llm(scratchpad: List[Message]) -> str:
    """Formats the scratchpad content into a string for the LLM."""
    conversation = "\n".join([str(message) for message in scratchpad])
    return conversation

Here we define another helper function that leverages our `pretty_print` module to show each step of the ReAct loop in a beautiful and colored way.

In [12]:
def pretty_print_message(message: Message, turn: int, max_turns: int, header_color: str = pretty_print.Color.YELLOW, is_forced_final_answer: bool = False) -> None:
    if not is_forced_final_answer:
        title = f"{message.role.value.capitalize()} (Turn {turn}/{max_turns}):"
    else:
        title = f"{message.role.value.capitalize()} (Forced):"

    pretty_print.wrapped(
        text=message.content,
        title=title,
        header_color=header_color,
    )

We can now implement the ReAct loop.

In [13]:
def react_agent_loop(initial_question: str, max_turns: int = 5, verbose: bool = False) -> str:
    """
    Implements the main ReAct (Thought -> Action -> Observation) control loop.
    Uses a unified message class for the scratchpad.
    """
    # The scratchpad stores the history of messages.
    scratchpad: List[Message] = []

    for turn in range(1, max_turns + 1):
        # If it's the first turn, add the user's question to the scratchpad
        if not scratchpad:
            user_message = Message(role=MessageRole.USER, content=initial_question)
            scratchpad.append(user_message)
            if verbose:
                pretty_print_message(user_message, turn, max_turns, pretty_print.Color.RESET)

        # Generate a thought based on the current scratchpad
        thought_content = generate_thought(format_scratchpad_for_llm(scratchpad))
        thought_message = Message(role=MessageRole.THOUGHT, content=thought_content)
        scratchpad.append(thought_message)
        if verbose:
            pretty_print_message(thought_message, turn, max_turns, pretty_print.Color.ORANGE)

        # Generate an action based on the current scratchpad
        action_name, action_params = generate_action(format_scratchpad_for_llm(scratchpad))

        # Handle the finish action
        if action_name == ACTION_FINISH:
            final_answer = action_params.text
            final_message = Message(role=MessageRole.FINAL_ANSWER, content=final_answer)
            scratchpad.append(final_message)
            if verbose:
                pretty_print_message(final_message, turn, max_turns, pretty_print.Color.CYAN)
            return final_answer

        # Handle a tool request action
        params_str = ", ".join([f"{k}='{v}'" for k, v in action_params.items()])
        action_content = f"{action_name}({params_str})"
        action_message = Message(role=MessageRole.TOOL_REQUEST, content=action_content)
        scratchpad.append(action_message)
        if verbose:
            pretty_print_message(action_message, turn, max_turns, pretty_print.Color.GREEN)

        # Run the action and get the observation
        observation_content = ""
        if action_name in TOOL_REGISTRY:
            tool_function = TOOL_REGISTRY[action_name]
            try:
                # Use ** to unpack the dictionary of parameters into function arguments
                observation_content = tool_function(**action_params)
            except Exception as e:
                observation_content = f"Error executing tool '{action_name}': {e}"
        else:
            available_tools_str = ", ".join(TOOL_REGISTRY.keys())
            observation_content = f"Error - Unknown action '{action_name}'. Available tools are [{available_tools_str}]."

        # Add the observation to the scratchpad
        observation_message = Message(role=MessageRole.OBSERVATION, content=observation_content)
        scratchpad.append(observation_message)
        if verbose:
            pretty_print_message(observation_message, turn, max_turns, pretty_print.Color.YELLOW)


        # Check if the maximum number of turns has been reached. If so, force generating a final answer
        if turn == max_turns:
            final_answer = generate_final_answer(format_scratchpad_for_llm(scratchpad))
            final_message = Message(role=MessageRole.FINAL_ANSWER, content=final_answer)
            scratchpad.append(final_message)
            if verbose:
                pretty_print_message(final_message, turn, max_turns, pretty_print.Color.CYAN, is_forced_final_answer=True)
            return final_answer

Let's test our ReAct agent with a simple factual question that requires a search:

In [17]:
# A straightforward question requiring a search.
question = "What is the capital of France?"
final_answer = react_agent_loop(question, max_turns=2, verbose=True)

----------------------------------------- User (Turn 1/2): -----------------------------------------
  What is the capital of France?
----------------------------------------------------------------------------------------------------
--------------------------------------- Thought (Turn 1/2): ---------------------------------------
  The user is asking a factual question about the capital of France. The next step is to retrieve this specific piece of information.
----------------------------------------------------------------------------------------------------
------------------------------------- Tool request (Turn 1/2): -------------------------------------
  search(query='capital of France')
----------------------------------------------------------------------------------------------------
------------------------------------- Observation (Turn 1/2): -------------------------------------
  Paris is the capital of France and is known for the Eiffel Tower.
------------------------

Now let's test with a more complex conceptual question that our mock search tool has knowledge about:

In [15]:
# A question about a concept the mock search tool might know.
question = "Can you explain the ReAct framework in AI?"
final_answer = react_agent_loop(question, max_turns=2, verbose=True)

----------------------------------------- User (Turn 1/2): -----------------------------------------
  Can you explain the ReAct framework in AI?
----------------------------------------------------------------------------------------------------
--------------------------------------- Thought (Turn 1/2): ---------------------------------------
  The user is asking for an explanation of the ReAct framework in AI. This requires retrieving factual information about the framework. I need to gather details such as its full name, core concept, how it operates (the iterative process), its advantages, and typical use cases. I will prioritize external information retrieval for all these details.
----------------------------------------------------------------------------------------------------
------------------------------------- Tool request (Turn 1/2): -------------------------------------
  search(query='ReAct framework AI')
----------------------------------------------------------------

Last, let's test with a question that our mock search tool doesn't have knowledge about:

In [16]:
# A question about a concept the mock search tool doesn't know.
question = "What is the capital of Italy?"
final_answer = react_agent_loop(question, max_turns=2, verbose=True)

----------------------------------------- User (Turn 1/2): -----------------------------------------
  What is the capital of Italy?
----------------------------------------------------------------------------------------------------
--------------------------------------- Thought (Turn 1/2): ---------------------------------------
  The user is asking a factual question about the capital of Italy. The next step is to retrieve this specific piece of information. The information needed is the name of Italy's capital city. The plan is to perform a targeted search query like 'capital of Italy' and extract the answer from reliable sources.
----------------------------------------------------------------------------------------------------
------------------------------------- Tool request (Turn 1/2): -------------------------------------
  search(query='capital of Italy')
----------------------------------------------------------------------------------------------------
------------------